In [1]:
import os
from google.colab import drive

mount_dir = '/content/gdrive/'
drive.mount(mount_dir)
wd = mount_dir+'My Drive/Work/UkNotUk/'
model_folder = wd+'model_checkpoints'
results_folder = wd+'results/'

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
os.chdir(wd+'notebooks/attention-module')

In [3]:
!ls

checkpoints  outputs		ResNetCBAMGradcam.ipynb
LICENSE      README.md		scripts
MODELS	     RESNET50_CBAM.pth	train_imagenet.py


In [4]:
#imports
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_fscore_support, confusion_matrix, accuracy_score
from torchvision import transforms 
#from torchvision.models import resnet50
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch import nn, optim
import cv2

In [5]:
#GET DATA
datafolder = wd+'data/train/'

transformations = transforms.Compose([transforms.Resize((224, 224)), 
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
dataset = ImageFolder(datafolder,transform=transformations) 

In [6]:
loss_weights = torch.Tensor([np.sum(np.array(dataset.targets) == 0), np.sum(np.array(dataset.targets) == 1)])#/len(dataset.targets)
loss_weights

tensor([5406., 2350.])

In [7]:
#split training set into training and validation
from torch.utils.data.sampler import SubsetRandomSampler

#get the length of the dataset
dataset_size = len(dataset)

#get a list of indices
indices = list(range(dataset_size))

#shuffle the indices in place
np.random.shuffle(indices)

#pick training proportion
train_size, valid_size, test_size = int(0.7*dataset_size), int(0.2*dataset_size), int(0.1*dataset_size)

#separate indices
train_indices = indices[:train_size]
valid_indices = indices[train_size:train_size+valid_size]
test_indices = indices[train_size+valid_size:]

#create samplers
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)
test_sampler = SubsetRandomSampler(test_indices)

In [8]:
#split training set into training and validation
from torch.utils.data.sampler import SubsetRandomSampler

#get the length of the dataset
dataset_size = len(dataset)

#get a list of indices
indices = list(range(dataset_size))

#shuffle the indices in place
np.random.shuffle(indices)

#pick training proportion
train_size, valid_size, test_size = int(0.7*dataset_size), int(0.2*dataset_size), int(0.1*dataset_size)

#separate indices
train_indices = indices[:train_size]
valid_indices = indices[train_size:train_size+valid_size]
test_indices = indices[train_size+valid_size:]

#create samplers
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)
test_sampler = SubsetRandomSampler(test_indices)

In [9]:
#Dataloaders
train_loader = DataLoader(dataset, batch_size=16, num_workers=3, sampler=train_sampler, pin_memory=True)
valid_loader = DataLoader(dataset, batch_size=16, num_workers=3, sampler=valid_sampler, pin_memory=True)
test_loader = DataLoader(dataset, batch_size=16, num_workers=3, sampler=test_sampler, pin_memory=True)

In [10]:
#device settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
if torch.cuda.is_available():
  device_count = torch.cuda.device_count()
  print(device_count)

1


In [12]:
from MODELS.model_resnet import *

In [13]:
class myResNet50(nn.Module):
  def __init__(self, should_hook=False, att_type=None, attn_pretrained=False, vanilla_pretrained=False):
    super(myResNet50, self).__init__()

    self.should_hook = should_hook
    pretained_cbam_model_path = './RESNET50_CBAM.pth'

    if vanilla_pretrained:
      print('using resnet 50 pretrained model')

    if attn_pretrained:
      print('using CBAM pretrained model')

    pretrainedmodel = ResidualNet('ImageNet', 50, 1000, att_type, pretrained=vanilla_pretrained, modelpath=None)
    #self.avgpool = pretrainedmodel.avgpool
    
    print(pretrainedmodel)
    #resnet50(pretrained=True)
    if attn_pretrained:
        if os.path.isfile(pretained_cbam_model_path):
            print("=> loading checkpoint '{}'".format(pretained_cbam_model_path))
            checkpoint = torch.load(pretained_cbam_model_path)
            for key in list(checkpoint['state_dict'].keys()):
              newkey = key.replace('module.','')
              checkpoint['state_dict'][newkey] = checkpoint['state_dict'].pop(key)
            #args.start_epoch = checkpoint['epoch']
            #best_prec1 = checkpoint['best_prec1']
            pretrainedmodel.load_state_dict(checkpoint['state_dict'])
            #if 'optimizer' in checkpoint:
            #    optimizer.load_state_dict(checkpoint['optimizer'])
            print("Loaded checkpoint !!")
        else:
            print("=> no checkpoint found at '{}'".format(pretained_cbam_model_path))


    
    #freezes weights
    if not self.should_hook:
      print("Freezing params...")
      for param in pretrainedmodel.parameters():
        param.requires_grad = False

    self.base = nn.Sequential(*list(pretrainedmodel.children())[:-2])
    self.avgpool = pretrainedmodel.avgpool
    

    #final layer
    infeatures = pretrainedmodel.fc.in_features
    self.fc = nn.Linear(infeatures, 2)

    # placeholder for the gradients
    self.gradients = None
    
  # method for the gradient extraction
  def get_activations_gradient(self):
    return self.gradients
  
  # method for the activation exctraction
  def get_activations(self, x):
    return self.base(x)

  # hook for the gradients of the activations
  def activations_hook(self, grad):
    self.gradients = grad


  def forward(self, x):
    #x.requires_grad = True
    x = self.base(x)
    
    # register the hook
    if self.should_hook:
      h = x.register_hook(self.activations_hook)
    
    # apply the remaining pooling (optional) and final layer
    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    out = self.fc(x)
    return out





In [14]:
model = myResNet50(should_hook=True, att_type='CBAM', attn_pretrained=True, vanilla_pretrained=False)

if torch.cuda.is_available():
  device_count = torch.cuda.device_count()
  model = torch.nn.DataParallel(model, device_ids=list(range(device_count)))

model.to(device)
#print(model)

model_name = 'jan11_test_cbam_uk_not_uk_classifier'

using CBAM pretrained model
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, ker

In [15]:
print(model_name)
print(model)

jan11_test_cbam_uk_not_uk_classifier
DataParallel(
  (module): myResNet50(
    (base): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      

In [16]:
# get the number of model parameters
print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))

Number of model parameters: 26044722


In [17]:
#model settings
criterion = nn.CrossEntropyLoss(weight=loss_weights.to(device))

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.module.fc.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)

In [18]:
from torch.nn import functional as F

def get_confusion_matrix(y_pred, y_true):
    assert y_true.ndim == 1
    assert y_pred.ndim == 1 or y_pred.ndim == 2
    
    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)
        
    
    tp = (y_true * y_pred).sum().to(torch.float32)
    tn = ((1 - y_true) * (1 - y_pred)).sum().to(torch.float32)
    fp = ((1 - y_true) * y_pred).sum().to(torch.float32)
    fn = (y_true * (1 - y_pred)).sum().to(torch.float32)

    return np.array([tp, tn, fp, fn])

def get_metrics(tp, tn, fp, fn, epsilon=1e-7):
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    f1 = 2* (precision*recall) / (precision + recall + epsilon)

    return precision, recall, f1

In [19]:
#train network

# number of epochs to train the model
n_epochs = 30

valid_loss_min = np.Inf # track change in validation loss

#result files

train_results = results_folder+model_name+'_tr.csv'
val_results = results_folder+model_name+'_val.csv'

with open(train_results, 'a') as train_result:
    header = ",".join(['loss', 'accuracy', 'tn', 'fp', 'fn', 'tp', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1', 'count_0', 'count_1','auroc','\n'])
    train_result.write(header)

with open(val_results, 'a') as val_result:
    header = ",".join(['loss', 'accuracy', 'tn', 'fp', 'fn', 'tp', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1', 'count_0', 'count_1','auroc','saved','\n'])
    val_result.write(header)

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0

    # keep track of training and validation outputs
    tr_y_pred, tr_y_target = np.array([]), np.array([])
    val_y_pred, val_y_target = np.array([]), np.array([])

    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model.forward(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)

        #save data for metrics
        predictions = torch.exp(output)
      
        _, top_classes = predictions.topk(1,dim=1)

        tr_y_target = np.concatenate((tr_y_target,target.cpu()))
        tr_y_pred = np.concatenate((tr_y_pred,top_classes.squeeze(1).cpu()))
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        data, target = data.to(device), target.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model.forward(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)

        #save data for metrics
        predictions = torch.exp(output)
      
        _, top_classes = predictions.topk(1,dim=1)

        val_y_target = np.concatenate((val_y_target,target.cpu()))
        val_y_pred = np.concatenate((val_y_pred,top_classes.squeeze(1).cpu()))
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)

    # calulate metrics
    with open(train_results, 'a') as train_result:
      a, b = confusion_matrix(tr_y_target, tr_y_pred)
      tn, fp, fn, tp = *a, *b
      precision, recall, f1, support = precision_recall_fscore_support(tr_y_target, tr_y_pred)
      precision_0, precision_1, recall_0, recall_1, f1_0, f1_1, count_0, count_1 = *precision, *recall, *f1, *support
      accuracy = accuracy_score(tr_y_target, tr_y_pred)
      auroc = roc_auc_score(tr_y_target, tr_y_pred)
      line = ",".join([str(train_loss), str(accuracy), str(tn), str(fp), str(fn), str(tp), str(precision_0), str(precision_1), str(recall_0), str(recall_1), str(f1_0), str(f1_1), str(count_0), str(count_1), str(auroc), '\n'])
      train_result.write(line)

    with open(val_results, 'a') as val_result:
      a, b = confusion_matrix(val_y_target, val_y_pred)
      tn, fp, fn, tp = *a, *b
      precision, recall, f1, support = precision_recall_fscore_support(val_y_target, val_y_pred)
      precision_0, precision_1, recall_0, recall_1, f1_0, f1_1, count_0, count_1 = *precision, *recall, *f1, *support
      accuracy = accuracy_score(val_y_target, val_y_pred)
      auroc = roc_auc_score(val_y_target, val_y_pred)
      line = ",".join([str(valid_loss), str(accuracy), str(tn), str(fp), str(fn), str(tp), str(precision_0), str(precision_1), str(recall_0), str(recall_1), str(f1_0), str(f1_1), str(count_0), str(count_1), str(auroc), str(valid_loss <= valid_loss_min),'\n'])
      val_result.write(line)
      

        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))
    
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving updated model ...'.format(valid_loss_min, valid_loss))
        torch.save(model.state_dict(), model_folder+'/'+model_name+'.pt')
        valid_loss_min = valid_loss


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 1 	Training Loss: 0.218352 	Validation Loss: 0.091972
Validation loss decreased (inf --> 0.091972).  Saving updated model ...


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 2 	Training Loss: 0.150896 	Validation Loss: 0.119357


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


...

Epoch: 28 	Training Loss: 0.110105 	Validation Loss: 0.119785


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 29 	Training Loss: 0.079506 	Validation Loss: 0.074173


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 30 	Training Loss: 0.126288 	Validation Loss: 0.082533


In [37]:
from sklearn.metrics import roc_curve,average_precision_score, roc_auc_score, accuracy_score

fpr, tpr, _ = roc_curve(tr_y_target, tr_y_pred)
roc_auc = auc(fpr, tpr)
print(roc_auc)
print("average_precision_score : " + str(average_precision_score(tr_y_target, tr_y_pred)))
print("roc_auc_score : " + str(roc_auc_score(tr_y_target, tr_y_pred)))
print("accuracy : " + str(accuracy_score(tr_y_target, tr_y_pred)))

0.9401181585699138
average_precision_score : 0.8875409760074146
roc_auc_score : 0.9401181585699138
accuracy : 0.9561613556824461


In [33]:
#testing model

model = myResNet50(should_hook=False,att_type='CBAM', attn_pretrained=False)

if torch.cuda.is_available():
  device_count = torch.cuda.device_count()
  model = torch.nn.DataParallel(model, device_ids=list(range(device_count)))

model.load_state_dict(torch.load(model_folder+'/'+model_name+'.pt'))

model.eval()
test_accuracy = 0.0

y_pred, y_target = np.array([]), np.array([])

test_results = results_folder+model_name+'_test.csv'

with torch.no_grad():
  for test_images, test_labels in test_loader:
      test_images, test_labels = test_images.to(device), test_labels.to(device)
      out = model(test_images)
      predictions = torch.exp(out)
      
      _, top_classes = predictions.topk(1,dim=1)

      y_target = np.concatenate((y_target,test_labels.cpu()))
      y_pred = np.concatenate((y_pred,top_classes.squeeze(1).cpu()))
      

with open(test_results, 'a') as test_result:
  header = ",".join(['accuracy', 'tn', 'fp', 'fn', 'tp', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1', 'count_0', 'count_1','auroc','\n'])
  a, b = confusion_matrix(y_target, y_pred)
  tn, fp, fn, tp = *a, *b
  precision, recall, f1, support = precision_recall_fscore_support(y_target, y_pred)
  precision_0, precision_1, recall_0, recall_1, f1_0, f1_1, count_0, count_1 = *precision, *recall, *f1, *support
  accuracy = accuracy_score(y_target, y_pred)
  auroc = roc_auc_score(y_target, y_pred)
  line = ",".join([str(accuracy), str(tn), str(fp), str(fn), str(tp), str(precision_0), str(precision_1), str(recall_0), str(recall_1), str(f1_0), str(f1_1), str(count_0), str(count_1), str(auroc), '\n'])
  test_result.write(header)
  test_result.write(line)
  print(f"Results in {test_results}")

print(f'Test Accuracy: {accuracy*100:.6f}%')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Results in /content/gdrive/My Drive/Work/UkNotUk/results/jan11_test_cbam_uk_not_uk_classifier_test.csv
Test Accuracy: 98.453608%


## Visualise!!

In [25]:

def load_model(model_name, model=myResNet50(should_hook=True, att_type='CBAM', attn_pretrained=False)):

    if torch.cuda.is_available():
      device_count = torch.cuda.device_count()
      model = torch.nn.DataParallel(model, device_ids=list(range(device_count)))

    model.load_state_dict(torch.load(model_folder+'/'+model_name+'.pt'))
    model.eval()
    model = model.to(device)

    return model
    
def get_image_results(cam_model, image_file = './testimages/3946917.jpg'):
    img = plt.imread(image_file)
    img = transforms.ToPILImage()(img)
    img = transformations(img)
    img = img.unsqueeze(0)
    img = img.to(device)
    res = cam_model.forward(img)
    pred = res.argmax(dim=1)
    reses.append(res)
    res[:,pred.item()].backward()

    return img, res, pred, cam_model

def get_heatmap(cam_model, img, show_heatmap=False):
    # pull the gradients out of the model
    gradients = cam_model.module.get_activations_gradient()

    # pool the gradients across the channels
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])

    # get the activations of the last convolutional layer
    activations = cam_model.module.get_activations(img).detach()

    # weight the channels by corresponding gradients
    for i in range(512):
        activations[:, i, :, :] *= pooled_gradients[i]

    # average the channels of the activations
    heatmap = torch.mean(activations, dim=1).squeeze()

    # relu on top of the heatmap
    # expression (2) in https://arxiv.org/pdf/1610.02391.pdf
    heatmap = heatmap.cpu()

    heatmap = np.maximum(heatmap, 0)

    # normalize the heatmap
    heatmap /= torch.max(heatmap)
    
    if show_heatmap:
        # draw the heatmap
        plt.matshow(heatmap.squeeze())

    heatmap = heatmap.numpy()
    
    return heatmap

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [26]:
outputs = './outputs'
hmapss = []
def generate_superimposition(image_file, heatmap, model_short ='map'):
    if not os.path.exists(outputs):
        os.mkdir(outputs)
    else:
        res_folder = outputs +'/'+model_short
        if not os.path.exists(res_folder):
            os.mkdir(res_folder)
    
    filename = image_file.split('/')[-1]
    res_file = os.path.join(res_folder, filename.split('.jpg')[0]+'_'+model_short+'.jpg')
    img = cv2.imread(image_file)
    print('img shapee',img.shape)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * 0.4 + img
    cv2.imwrite(res_file, superimposed_img)
    return image_file,res_file
    
def show_superimposition(image_file, res_file):
    rimg = plt.imread(res_file)
    oimg = plt.imread(image_file)
    fig, ax = plt.subplots(1,2,figsize=(10,10))
    plt.figure(figsize=(20,20))
    ax[0].imshow(oimg)
    ax[1].imshow(rimg)
    plt.show()
    
def get_gradcam(cam_model, image_file, outputs = './outputs'):
    img, res, pred, cam_model = get_image_results(cam_model, image_file)
    heatmap = get_heatmap(cam_model, img)
    image_file, res_file = generate_superimposition(image_file, heatmap, model_short)
    return image_file, res_file

def show_superimpositions(image_pairs, title="Gradcam results"):
    fig, ax = plt.subplots(len(image_pairs),2,figsize=(90,90))
    row = 0
    for image_file, res_file in image_pairs:
        oimg = plt.imread(image_file)
        rimg = plt.imread(res_file)
        
        
        ax[row, 0].imshow(oimg)
        ax[row, 1].imshow(rimg)
        
        row+=1
    plt.tight_layout() 
    plt.suptitle(title)
    plt.show() 

In [27]:
#get test images
uk = wd+'data/val/uk'
notuk = wd+'data/val/not_uk'
uk_images = [os.path.join(uk,f) for f in os.listdir(uk) if f.endswith('.jpg')]
notuk_images = [os.path.join(notuk,f) for f in os.listdir(notuk) if f.endswith('.jpg')]

In [28]:
model_short = 'jan11fixedcbam'
cam_model = load_model(model_name)

In [29]:
uk_images = uk_images[:2]
notuk_images = notuk_images[:2]

In [30]:
uk_gradcam = [get_gradcam(cam_model, image_file) for image_file in uk_images]
notuk_gradcam = [get_gradcam(cam_model, image_file) for image_file in notuk_images]

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


hshapee torch.Size([7, 7])
img shapee (480, 640, 3)
hshapee torch.Size([7, 7])
img shapee (480, 640, 3)
hshapee torch.Size([7, 7])
img shapee (640, 480, 3)
hshapee torch.Size([7, 7])
img shapee (640, 480, 3)


In [31]:
#with cbam
show_superimpositions(uk_gradcam[:2], 'UK Images')

Output hidden; open in https://colab.research.google.com to view.

In [31]:
#without cbam
show_superimpositions(uk_gradcam[:2], 'UK Images')

Output hidden; open in https://colab.research.google.com to view.

In [32]:
#with cbam
show_superimpositions(notuk_gradcam[:2], 'NOT UK Images')

Output hidden; open in https://colab.research.google.com to view.

In [32]:
#without cbam
show_superimpositions(notuk_gradcam[:2], 'NOT UK Images')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print(model)

DataParallel(
  (module): myResNet50(
    (base): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (2): AdaptiveAvgPool2d(output_size=(1, 1))
      (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): ReLU(inplace=True)
      (5): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=